# Watershed Preservation Opportunity Map Model

## A GIS-based Decision Support System for Targeted Watershed Conservation

This notebook builds a comprehensive model that overlays water quality data with land use, impervious cover, and environmental justice layers to identify where watershed preservation would have the greatest human benefit.

### Key Innovation
- Converts water quality chemistry into spatial intelligence
- Prioritizes watershed preservation through quantifiable, local insights
- Combines water data with EJ layers to identify high vulnerability neighborhoods
- Creates a **Water Stress from Land Use Score** - a unified metric for conservation decisions

## 1. Environment Setup and Dependencies

In [3]:
# Core data processing
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Geospatial analysis
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pyproj
from pyproj import Transformer
import rasterio
from rasterio.mask import mask
from rasterstats import zonal_stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFE
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from scipy import stats
from scipy.spatial import distance
from scipy.interpolate import griddata
from scipy.stats import zscore

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("Environment ready. All dependencies loaded.")
print(f"GeoPandas version: {gpd.__version__}")
print(f"Rasterio version: {rasterio.__version__}")

Environment ready. All dependencies loaded.
GeoPandas version: 1.0.1
Rasterio version: 1.4.3


## 2. Data Loading and Preprocessing

### 2.1 Load Water Quality Dataset

In [5]:
# Load water quality data
# Replace with your actual data path
water_quality_df = pd.read_csv('sample_water_quality.csv')

# Display basic information
print(f"Dataset shape: {water_quality_df.shape}")
print(f"\nColumns: {water_quality_df.columns.tolist()}")
print(f"\nData types:")
print(water_quality_df.dtypes)
print(f"\nFirst few rows:")
water_quality_df.head()

Dataset shape: (12596, 48)

Columns: ['Unnamed: 0', 'Record_ID', 'Database_Version', 'Data_Year', 'Quality_Control_Status', 'Quality_Control_Date', 'Project_Name', 'Watershed_Name', 'Water_System_Code', 'Water_Body_Name', 'Station_ID', 'Station_Year_ID', 'Location_Description', 'Station_Type', 'River_Mile_Point', 'River_Mile_Point_Numeric', 'Latitude', 'Longitude', 'Field_Sheet_Code', 'Survey_Type', 'Sample_ID', 'Sample_Date', 'Sample_Date_Formatted', 'Sample_Time', 'Sample_Time_Formatted', 'Flow_Condition', 'Sample_Depth_m', 'Sample_Depth_Numeric', 'Depth_Qualifier_Code', 'Temperature_C', 'Temperature_C_Numeric', 'Temperature_Qualifier_Code', 'pH_Level', 'pH_Level_Numeric', 'pH_Qualifier_Code', 'Specific_Conductivity_uS_cm', 'Specific_Conductivity_Numeric', 'Conductivity_Qualifier_Code', 'Total_Dissolved_Solids_mg_L', 'Total_Dissolved_Solids_Numeric', 'TDS_Qualifier_Code', 'Dissolved_Oxygen_mg_L', 'Dissolved_Oxygen_Numeric', 'Dissolved_Oxygen_Qualifier_Code', 'Dissolved_Oxygen_Saturat

,Unnamed: 0,Record_ID,Database_Version,Data_Year,Quality_Control_Status,Quality_Control_Date,Project_Name,Watershed_Name,Water_System_Code,Water_Body_Name,Station_ID,Station_Year_ID,Location_Description,Station_Type,River_Mile_Point,River_Mile_Point_Numeric,Latitude,Longitude,Field_Sheet_Code,Survey_Type,Sample_ID,Sample_Date,Sample_Date_Formatted,Sample_Time,Sample_Time_Formatted,Flow_Condition,Sample_Depth_m,Sample_Depth_Numeric,Depth_Qualifier_Code,Temperature_C,Temperature_C_Numeric,Temperature_Qualifier_Code,pH_Level,pH_Level_Numeric,pH_Qualifier_Code,Specific_Conductivity_uS_cm,Specific_Conductivity_Numeric,Conductivity_Qualifier_Code,Total_Dissolved_Solids_mg_L,Total_Dissolved_Solids_Numeric,TDS_Qualifier_Code,Dissolved_Oxygen_mg_L,Dissolved_Oxygen_Numeric,Dissolved_Oxygen_Qualifier_Code,Dissolved_Oxygen_Saturation_Percent,Dissolved_Oxygen_Saturation_Numeric,DO_Saturation_Qualifier_Code,Result_Comments
0,6767,112276,Extracted from WPP_WQData_2005-2020.mdb on 6/3...,2009,QC4,2011-12-15 13:05:31,Cape Cod (2009),Cape Cod,9661650,Herring River,W1916,W1916,"[Bound Brook Island Road, Wellfleet]",River/Stream,--,NaN,41.953519,-70.057176,09-D503-02,River,96-0465,9/2/2009,2009-09-02,11:26:06 AM,11:26:06,Flowing,0.3,0.3,NaN,14.1,14.1,NaN,5.3,5.3,NaN,205,205.0,NaN,131,131.0,NaN,4.4,4.4,NaN,43,43.0,NaN,NaN
1,6138,111647,Extracted from WPP_WQData_2005-2020.mdb on 6/3...,2009,QC4,2011-12-15 13:05:31,Narragansett-Mt. Hope Bay (2009),Narragansett Bay (Shore),5334025,Runnins River,W0651,Run1,"[School Street, Seekonk]",River/Stream,1.1,1.100,41.788377,-71.329520,09-H509-08,River,53-0809,8/24/2009,2009-08-24,11:31:01 AM,11:31:01,Flowing,0.7,0.7,NaN,22.8,22.8,u,6.5,6.5,NaN,804,804.0,u,515,515.0,u,4.2,4.2,NaN,50,50.0,NaN,NaN
2,11159,118461,Extracted from WPP_WQData_2005-2020.mdb on 6/3...,2006,QC4,2017-10-11 09:35:30,SMART: French & Quinebaug (2006),Quinebaug,4128875,Quinebaug River,W0601,QR00,"[Holland Road bridge, Sturbridge]",River/Stream,15.202,15.202,42.109561,-72.118569,06-J004-01,River,SM-1803,7/19/2006,2006-07-19,8:53:03 AM,08:53:03,Flowing,0.2,0.2,NaN,27.4,27.4,NaN,7.0,7.0,NaN,109,109.0,NaN,71,71.0,NaN,7.2,7.2,NaN,92,92.0,NaN,NaN
3,10384,117686,Extracted from WPP_WQData_2005-2020.mdb on 6/3...,2006,QC4,2017-10-11 09:35:30,Concord (2006),Concord (SuAsCo),8246775,Assabet River,W1473,AS08,"[Robin Hill Street bridge, Marlborough (approx...",River/Stream,--,NaN,42.346501,-71.614603,06-A501-02,River,82-0323,6/5/2006,2006-06-05,9:00:31 AM,09:00:31,Flowing,0.0,0.0,i,15.1,15.1,NaN,--,NaN,NaN,--,NaN,NaN,--,NaN,NaN,7.3,7.3,NaN,74,74.0,NaN,NaN
4,5513,126923,Extracted from WPP_WQData_2005-2020.mdb on 6/3...,2011,QC4,2019-06-19 10:28:42,SMART: Chicopee (2011),Chicopee,3626500,Ware River,W0494,CBG,[south of Route 122 at weir downstream of Shaf...,River/Stream,30.29,30.290,42.391214,-72.064555,11-G004-02,River,SM-3709,8/30/2011,2011-08-30,9:45:31 AM,09:45:31,Flowing,1.8,1.8,NaN,18.4,18.4,NaN,5.7,5.7,NaN,54,54.0,NaN,35,35.0,NaN,8.1,8.1,NaN,87,87.0,NaN,NaN


In [6]:
# Map actual column names to simplified names for analysis
column_mapping = {
    'pH_Level_Numeric': 'pH',
    'Specific_Conductivity_Numeric': 'conductivity',
    'Temperature_C_Numeric': 'temperature',
    'Total_Dissolved_Solids_Numeric': 'TDS',
    'Dissolved_Oxygen_Numeric': 'DO'
}

# Create simplified column names
for old_col, new_col in column_mapping.items():
    if old_col in water_quality_df.columns:
        water_quality_df[new_col] = pd.to_numeric(water_quality_df[old_col], errors='coerce')

# Parse datetime if needed
if 'Sample_Date_Formatted' in water_quality_df.columns:
    water_quality_df['date'] = pd.to_datetime(water_quality_df['Sample_Date_Formatted'], errors='coerce')
    water_quality_df['year'] = water_quality_df['date'].dt.year
    water_quality_df['month'] = water_quality_df['date'].dt.month
    water_quality_df['season'] = water_quality_df['date'].dt.month % 12 // 3 + 1
elif 'Sample_Date' in water_quality_df.columns:
    water_quality_df['date'] = pd.to_datetime(water_quality_df['Sample_Date'], errors='coerce')
    water_quality_df['year'] = water_quality_df['date'].dt.year
    water_quality_df['month'] = water_quality_df['date'].dt.month
    water_quality_df['season'] = water_quality_df['date'].dt.month % 12 // 3 + 1

# Key water quality parameters for analysis
key_params = ['pH', 'conductivity', 'temperature', 'TDS', 'DO']

# Ensure all key parameters are numeric (already done above, but double-check)
for param in key_params:
    if param in water_quality_df.columns:
        water_quality_df[param] = pd.to_numeric(water_quality_df[param], errors='coerce')

print("Water quality data preprocessed successfully.")
print(f"Available parameters: {[p for p in key_params if p in water_quality_df.columns]}")

Water quality data preprocessed successfully.
Available parameters: ['pH', 'conductivity', 'temperature', 'TDS', 'DO']


In [7]:
water_quality_df.columns = water_quality_df.columns.str.lower()

### 2.2 Station-Level Aggregation and Stress Metrics

In [9]:
def calculate_station_metrics(df, station_col='station_id', params=key_params):
    """
    Calculate comprehensive water stress metrics for each monitoring station
    """
    metrics = []
    
    # Handle case-insensitive column names
    station_col_lower = station_col.lower()
    df_lower = df.copy()
    df_lower.columns = df_lower.columns.str.lower()
    
    # Find the actual station column name
    if station_col_lower in df_lower.columns:
        station_col_actual = df_lower.columns[df_lower.columns == station_col_lower][0]
    else:
        # Try to find station_id or similar
        station_col_actual = [col for col in df_lower.columns if 'station' in col.lower()][0] if any('station' in col.lower() for col in df_lower.columns) else station_col_lower
    
    for station in df_lower[station_col_actual].unique():
        station_data = df_lower[df_lower[station_col_actual] == station]
        
        station_metrics = {
            'station_id': station,
            'n_observations': len(station_data),
            'latitude': station_data['latitude'].iloc[0] if 'latitude' in station_data.columns else None,
            'longitude': station_data['longitude'].iloc[0] if 'longitude' in station_data.columns else None,
        }
        
        for param in params:
            param_lower = param.lower()
            if param_lower in station_data.columns:
                param_data = station_data[param_lower].dropna()
                
                if len(param_data) > 0:
                    # Basic statistics
                    station_metrics[f'{param}_mean'] = param_data.mean()
                    station_metrics[f'{param}_std'] = param_data.std()
                    station_metrics[f'{param}_cv'] = param_data.std() / param_data.mean() if param_data.mean() != 0 else 0
                    
                    # Percentiles for anomaly detection
                    station_metrics[f'{param}_p25'] = param_data.quantile(0.25)
                    station_metrics[f'{param}_median'] = param_data.median()
                    station_metrics[f'{param}_p75'] = param_data.quantile(0.75)
                    station_metrics[f'{param}_p95'] = param_data.quantile(0.95)
                    
                    # Extreme value frequency
                    upper_threshold = param_data.quantile(0.9)
                    lower_threshold = param_data.quantile(0.1)
                    station_metrics[f'{param}_extreme_freq'] = (
                        ((param_data > upper_threshold) | (param_data < lower_threshold)).sum() / len(param_data)
                    )
                    
                    # Trend analysis if temporal data available
                    if 'date' in station_data.columns and len(param_data) > 10:
                        # Sort by date for trend analysis
                        station_data_sorted = station_data.sort_values('date')
                        param_data_sorted = station_data_sorted[param_lower].dropna()
                        if len(param_data_sorted) > 10:
                            x = np.arange(len(param_data_sorted))
                            slope, intercept, r_value, p_value, std_err = stats.linregress(x, param_data_sorted.values)
                            station_metrics[f'{param}_trend'] = slope
                            station_metrics[f'{param}_trend_pvalue'] = p_value
        
        metrics.append(station_metrics)
    
    return pd.DataFrame(metrics)

# Calculate station-level metrics
station_metrics_df = calculate_station_metrics(water_quality_df)
print(f"Calculated metrics for {len(station_metrics_df)} stations")
print(f"\nStation metrics columns: {station_metrics_df.columns.tolist()[:10]}...")
station_metrics_df.head()

Calculated metrics for 1320 stations

Station metrics columns: ['station_id', 'n_observations', 'latitude', 'longitude', 'pH_mean', 'pH_std', 'pH_cv', 'pH_p25', 'pH_median', 'pH_p75']...


,station_id,n_observations,latitude,longitude,pH_mean,pH_std,pH_cv,pH_p25,pH_median,pH_p75,pH_p95,pH_extreme_freq,conductivity_mean,conductivity_std,conductivity_cv,conductivity_p25,conductivity_median,conductivity_p75,conductivity_p95,conductivity_extreme_freq,temperature_mean,temperature_std,temperature_cv,temperature_p25,temperature_median,temperature_p75,temperature_p95,temperature_extreme_freq,TDS_mean,TDS_std,TDS_cv,TDS_p25,TDS_median,TDS_p75,TDS_p95,TDS_extreme_freq,DO_mean,DO_std,DO_cv,DO_p25,DO_median,DO_p75,DO_p95,DO_extreme_freq,pH_trend,pH_trend_pvalue,conductivity_trend,conductivity_trend_pvalue,temperature_trend,temperature_trend_pvalue,TDS_trend,TDS_trend_pvalue,DO_trend,DO_trend_pvalue
0,W1916,8,41.953519,-70.057176,5.857143,0.386683,0.066019,5.600,5.90,6.15,6.27,0.285714,204.857143,28.933421,0.141237,202.00,206.0,221.00,232.70,0.285714,15.625000,2.081037,0.133186,14.1,15.25,15.85,19.020,0.250000,131.000000,18.681542,0.142607,129.00,132.0,141.50,148.90,0.285714,5.425000,1.187735,0.218937,4.400,5.0,6.500,6.995,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W0651,8,41.788377,-71.329520,6.737500,0.106066,0.015743,6.700,6.80,6.80,6.80,0.125000,540.250000,200.335255,0.370820,416.75,465.5,621.75,856.00,0.250000,19.000000,3.730952,0.196366,16.2,18.50,22.20,23.580,0.250000,345.750000,128.091206,0.370473,266.75,298.0,398.00,547.50,0.250000,6.175000,1.398724,0.226514,5.375,6.2,7.175,7.895,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W0601,44,42.109561,-72.118569,6.590909,0.370304,0.056184,6.375,6.65,6.90,7.00,0.136364,116.250000,18.263097,0.157102,108.50,117.5,125.25,145.55,0.227273,12.747727,8.413955,0.660036,5.5,12.25,20.70,25.085,0.227273,75.837209,11.912239,0.157076,72.50,77.0,82.50,94.80,0.232558,10.561364,2.473226,0.234177,8.200,10.1,12.500,14.540,0.204545,0.005201,0.241240,-0.277590,0.204062,0.039049,0.700688,-0.222138,0.130712,-0.010987,0.712944
3,W1473,13,42.346501,-71.614603,7.020000,0.192354,0.027401,7.000,7.10,7.10,7.18,0.400000,586.000000,134.188301,0.228990,643.00,645.0,646.00,649.20,0.400000,18.676923,3.448467,0.184638,16.0,20.00,20.60,23.380,0.307692,379.600000,86.436682,0.227705,416.00,418.0,419.00,419.80,0.400000,7.538462,1.256674,0.166702,6.500,7.2,8.400,9.580,0.307692,NaN,NaN,NaN,NaN,0.166484,0.538479,NaN,NaN,-0.017033,0.864019
4,W0494,54,42.391214,-72.064555,6.273077,0.348732,0.055592,6.175,6.40,6.50,6.70,0.211538,88.596154,15.375381,0.173545,78.00,87.0,101.25,113.90,0.211538,12.864815,8.361506,0.649951,5.3,13.45,20.50,23.770,0.203704,57.519231,9.820927,0.170742,50.75,57.0,65.25,72.45,0.230769,10.598039,2.490983,0.235042,8.350,10.3,12.900,14.550,0.196078,0.006830,0.032622,-0.137668,0.337483,0.000606,0.993471,-0.094980,0.299824,0.004136,0.863500


## 3. GIS Data Integration

### 3.1 Convert Stations to GeoDataFrame

In [11]:
# Create GeoDataFrame from station metrics
geometry = [Point(xy) for xy in zip(station_metrics_df['longitude'], station_metrics_df['latitude'])]
stations_gdf = gpd.GeoDataFrame(station_metrics_df, geometry=geometry, crs='EPSG:4326')

# Transform to Massachusetts State Plane (meters) for accurate distance calculations
ma_crs = 'EPSG:26986'  # NAD83 / Massachusetts Mainland
stations_gdf = stations_gdf.to_crs(ma_crs)

print(f"Created GeoDataFrame with {len(stations_gdf)} monitoring stations")
print(f"CRS: {stations_gdf.crs}")

Created GeoDataFrame with 1320 monitoring stations
CRS: EPSG:26986


### 3.2 Load and Process GIS Layers

Note: These are placeholder functions. Replace with actual MassGIS data URLs/paths

In [13]:
def create_mock_impervious_layer(stations_gdf):
    """Create mock impervious surface data for demonstration"""
    bounds = stations_gdf.total_bounds
    # Create random polygons representing impervious areas
    mock_data = []
    np.random.seed(42)
    for i in range(100):
        x = np.random.uniform(bounds[0], bounds[2])
        y = np.random.uniform(bounds[1], bounds[3])
        size = np.random.uniform(100, 1000)
        poly = Point(x, y).buffer(size)
        mock_data.append({
            'geometry': poly,
            'imperv_pct': np.random.uniform(30, 95)
        })
    return gpd.GeoDataFrame(mock_data, crs=ma_crs)

def create_mock_landuse_layer(stations_gdf):
    """Create mock land use data for demonstration"""
    bounds = stations_gdf.total_bounds
    landuse_types = ['Forest', 'Urban', 'Agriculture', 'Wetland', 'Industrial']
    mock_data = []
    np.random.seed(43)
    for i in range(150):
        x = np.random.uniform(bounds[0], bounds[2])
        y = np.random.uniform(bounds[1], bounds[3])
        size = np.random.uniform(500, 5000)
        poly = Point(x, y).buffer(size)
        mock_data.append({
            'geometry': poly,
            'landuse': np.random.choice(landuse_types),
            'area_sq_m': poly.area
        })
    return gpd.GeoDataFrame(mock_data, crs=ma_crs)

def create_mock_watershed_layer(stations_gdf):
    """Create mock watershed boundaries for demonstration"""
    from scipy.spatial import Voronoi
    points = np.column_stack([stations_gdf.geometry.x, stations_gdf.geometry.y])
    # Create Voronoi polygons as mock watersheds
    vor = Voronoi(points)
    # Simplified: use station buffers as watersheds
    watersheds = []
    for idx, station in stations_gdf.iterrows():
        watersheds.append({
            'geometry': station.geometry.buffer(5000),  # 5km radius
            'watershed_id': f"WS_{idx}",
            'area_sq_km': np.pi * 25  # Approximate area
        })
    return gpd.GeoDataFrame(watersheds, crs=ma_crs)

def create_mock_ej_layer(stations_gdf):
    """Create mock EJ communities data for demonstration"""
    bounds = stations_gdf.total_bounds
    mock_data = []
    np.random.seed(44)
    for i in range(30):
        x = np.random.uniform(bounds[0], bounds[2])
        y = np.random.uniform(bounds[1], bounds[3])
        size = np.random.uniform(2000, 8000)
        poly = Point(x, y).buffer(size)
        mock_data.append({
            'geometry': poly,
            'ej_criteria': np.random.choice(['Income', 'Minority', 'Both']),
            'population': np.random.randint(1000, 50000),
            'vulnerability_score': np.random.uniform(0.3, 1.0)
        })
    return gpd.GeoDataFrame(mock_data, crs=ma_crs)

def create_mock_protected_layer(stations_gdf):
    """Create mock protected lands data for demonstration"""
    bounds = stations_gdf.total_bounds
    mock_data = []
    np.random.seed(45)
    for i in range(40):
        x = np.random.uniform(bounds[0], bounds[2])
        y = np.random.uniform(bounds[1], bounds[3])
        size = np.random.uniform(1000, 10000)
        poly = Point(x, y).buffer(size)
        mock_data.append({
            'geometry': poly,
            'protection_type': np.random.choice(['State Park', 'Conservation', 'Wildlife Refuge']),
            'area_hectares': poly.area / 10000
        })
    return gpd.GeoDataFrame(mock_data, crs=ma_crs)


In [ ]:
import geopandas as gpd
import os

def map_landuse_categories(shp_link_value):
    """
    Map shp_link values to standardized land use categories
    """
    if pd.isna(shp_link_value):
        return 'Other'
    
    shp_str = str(shp_link_value).lower()
    
    # Forest categories
    if any(x in shp_str for x in ['forest', 'wood', 'tree', 'deciduous', 'coniferous']):
        return 'Forest'
    # Urban categories
    elif any(x in shp_str for x in ['urban', 'residential', 'commercial', 'developed', 'impervious', 'building']):
        return 'Urban'
    # Industrial
    elif any(x in shp_str for x in ['industrial', 'manufacturing', 'warehouse']):
        return 'Industrial'
    # Wetland
    elif any(x in shp_str for x in ['wetland', 'marsh', 'swamp', 'bog']):
        return 'Wetland'
    # Agriculture
    elif any(x in shp_str for x in ['agriculture', 'agricultural', 'crop', 'farm', 'field', 'pasture']):
        return 'Agriculture'
    # Water
    elif any(x in shp_str for x in ['water', 'open water', 'river', 'lake', 'pond']):
        return 'Water'
    # Recreation
    elif any(x in shp_str for x in ['recreation', 'park', 'golf']):
        return 'Recreation'
    else:
        return 'Other'

def fetch_mass_dor_economic_data():
    """
    Fetch real economic data from Massachusetts DOR Municipal Databank
    
    Source: https://dls-gw.dor.state.ma.us/reports/rdPage.aspx?rdReport=Socioeconomic.MedHouseholdFamInc
    
    Returns:
        DataFrame with municipality income data
    """
    try:
        from mass_dor_economic_data import fetch_dor_income_data, load_dor_income_data_from_file
        import os
        
        # Try to load from file if available
        dor_file = None
        for f in ['ma_income_data.csv', 'ma_dor_income.csv', 'dor_socioeconomic.csv']:
            if os.path.exists(f):
                dor_file = f
                break
        
        if dor_file:
            print(f"Loading DOR data from file: {dor_file}")
            income_data = load_dor_income_data_from_file(dor_file)
        else:
            print("Using built-in MA DOR income data (2019 estimates)")
            income_data = fetch_dor_income_data()
        
        return income_data
    except ImportError:
        print("Warning: mass_dor_economic_data module not found. Using synthetic data.")
        return None
    except Exception as e:
        print(f"Warning: Could not load DOR data: {e}. Using synthetic data.")
        return None


def fetch_census_data_for_stations(stations_df, api_key=None, use_dor_data=True):
    """
    Fetch economic data for monitoring stations using MA DOR data or Census API.
    
    Priority: MA DOR data (municipality-level) > Census API (tract-level) > Synthetic proxies
    
    Args:
        stations_df: DataFrame with 'latitude', 'longitude', and optionally 'Location_Description'
        api_key: Optional Census API key
        use_dor_data: If True, try to use MA DOR municipal data first
    
    Returns:
        DataFrame with economic data merged to stations
    """
    import requests
    import time
    
    stations_with_econ = stations_df.copy()
    
    # Priority 1: Try MA DOR municipal data (most accurate for MA)
    if use_dor_data:
        try:
            from mass_dor_economic_data import join_economic_data_to_stations, fetch_dor_income_data
            
            print("Attempting to use MA DOR municipal income data...")
            income_data = fetch_dor_income_data()
            
            # Determine location columns
            lat_col = None
            lon_col = None
            location_col = None
            
            for col in ['latitude', 'Latitude', 'lat']:
                if col in stations_df.columns:
                    lat_col = col
                    break
            
            for col in ['longitude', 'Longitude', 'lon', 'lng']:
                if col in stations_df.columns:
                    lon_col = col
                    break
            
            for col in ['Location_Description', 'location_description', 'location', 'Location']:
                if col in stations_df.columns:
                    location_col = col
                    break
            
            if lat_col and lon_col:
                stations_with_econ = join_economic_data_to_stations(
                    stations_df,
                    income_data,
                    lat_col=lat_col,
                    lon_col=lon_col,
                    location_col=location_col
                )
                
                # Check if we got real data
                if stations_with_econ['median_household_income'].notna().sum() > 0:
                    print(f"✓ Successfully joined MA DOR economic data for {stations_with_econ['median_household_income'].notna().sum()} stations")
                    return stations_with_econ
                else:
                    print("  DOR data join returned no matches, falling back to synthetic data")
            else:
                print("  Missing coordinate columns, falling back to synthetic data")
        except Exception as e:
            print(f"  Could not use DOR data: {e}. Falling back to synthetic data.")
    
# Fallback: Synthetic proxies (original method)
    print("Using synthetic economic proxies based on location...")
    
    # Initialize stations_with_census BEFORE any conditionals
    stations_with_census = stations_with_econ.copy()
    
    
    
    
    # Find latitude/longitude columns (handle different case variations)
    lat_col_fallback = None
    lon_col_fallback = None
    for col in stations_with_census.columns:
        col_lower = col.lower()
        if col_lower in ['latitude', 'lat']:
            lat_col_fallback = col
        if col_lower in ['longitude', 'lon', 'lng']:
            lon_col_fallback = col
    
    if lat_col_fallback and lon_col_fallback:
    print("Using synthetic economic proxies based on location...")
    
    # Initialize stations_with_census from stations_with_econ
    
    # Find latitude/longitude columns (handle different case variations)
    lat_col_fallback = None
    lon_col_fallback = None
    for col in stations_with_census.columns:
        if col.lower() in ['latitude', 'lat']:
            lat_col_fallback = col
        if col.lower() in ['longitude', 'lon', 'lng']:
            lon_col_fallback = col
    
    if lat_col_fallback and lon_col_fallback:
        # Use lat/lon to create proxy economic indicators
        # Urban areas (higher lat, around -71) tend to have different characteristics
        # This is a simplified proxy - replace with actual Census API calls
        
        # Calculate distance from Boston (approximate urban center)
        boston_lat, boston_lon = 42.3601, -71.0589
        
        stations_with_census['dist_from_boston_km'] = np.sqrt(
            (stations_with_census[lat_col_fallback] - boston_lat)**2 + 
            (stations_with_census[lon_col_fallback] - boston_lon)**2
        ) * 111  # Approximate km conversion
        
        # Proxy economic indicators (correlate with distance from urban center)
        # Add realistic variation to create more diverse economic profiles
        # These would be replaced with actual Census API data
        
        # Base income varies by distance, with added noise for realism
        base_income = 45000 + 35000 * np.exp(-stations_with_census['dist_from_boston_km'] / 50)
        income_noise = np.random.normal(0, 8000, len(stations_with_census))  # Add variation
        stations_with_census['median_household_income'] = np.clip(
            base_income + income_noise,
            30000, 120000  # Realistic MA income range
        )
        
        # Add mean income (typically higher than median due to skew)
        stations_with_census['mean_household_income'] = stations_with_census['median_household_income'] * 1.15
        
        # Income inequality proxy (ratio of mean to median)
        stations_with_census['income_inequality_ratio'] = (
            stations_with_census['mean_household_income'] / stations_with_census['median_household_income']
        )
        
        # Poverty rate with variation
        base_poverty = 5 + 15 * np.exp(-stations_with_census['dist_from_boston_km'] / 40)
        poverty_noise = np.random.normal(0, 3, len(stations_with_census))
        stations_with_census['poverty_rate'] = np.clip(
            base_poverty + poverty_noise,
            2, 30  # Realistic poverty range
        )
        
        # Population density with variation
        base_density = 5000 * np.exp(-stations_with_census['dist_from_boston_km'] / 30)
        density_noise = np.random.normal(0, base_density * 0.3, len(stations_with_census))
        stations_with_census['population_density'] = np.clip(
            base_density + density_noise,
            50, 15000  # Realistic density range (rural to urban)
        )
        
        # Education with variation
        base_college = 30 + 25 * np.exp(-stations_with_census['dist_from_boston_km'] / 50)
        college_noise = np.random.normal(0, 8, len(stations_with_census))
        stations_with_census['percent_college_educated'] = np.clip(
            base_college + college_noise,
            15, 75  # Realistic education range
        )
        
        # Additional economic indicators
        # Unemployment rate (correlates with poverty)
        stations_with_census['unemployment_rate'] = np.clip(
            stations_with_census['poverty_rate'] * 0.6 + np.random.normal(2, 1.5, len(stations_with_census)),
            1, 12
        )
        
        # Median home value (correlates with income)
        stations_with_census['median_home_value'] = (
            stations_with_census['median_household_income'] * 3.5 + 
            np.random.normal(0, 50000, len(stations_with_census))
        ).clip(150000, 800000)
        
        # Per capita income (household income / average household size)
        avg_household_size = 2.3 + np.random.normal(0, 0.3, len(stations_with_census))
        stations_with_census['per_capita_income'] = (
            stations_with_census['median_household_income'] / avg_household_size
        )
        
        # Economic diversity index (proxy for economic stability)
        # Higher diversity = more stable economy
        income_cv = stations_with_census['median_household_income'].std() / stations_with_census['median_household_income'].mean()
        stations_with_census['economic_diversity_index'] = np.clip(
            1 - (income_cv / 2), 0.3, 0.9
        ) + np.random.normal(0, 0.1, len(stations_with_census))
        
        # Low-income households percentage
        stations_with_census['percent_low_income'] = np.clip(
            stations_with_census['poverty_rate'] * 1.5 + np.random.normal(0, 2, len(stations_with_census)),
            5, 40
        )
        
        # EJ proxy based on income and population density
        stations_with_census['ej_vulnerability_proxy'] = (
            (stations_with_census['poverty_rate'] / 25) * 0.4 +
            (1 - stations_with_census['median_household_income'] / 80000) * 0.3 +
            (stations_with_census['population_density'] / 5000) * 0.3
        ).clip(0, 1)
        
        stations_with_census['in_ej_community'] = (
            stations_with_census['ej_vulnerability_proxy'] > 0.5
        ).astype(int)
        
        print(f"✓ Created comprehensive economic indicators for {len(stations_with_census)} stations")
        print(f"  - Median income range: ${stations_with_census['median_household_income'].min():,.0f} - ${stations_with_census['median_household_income'].max():,.0f}")
        print(f"  - Mean income range: ${stations_with_census['mean_household_income'].min():,.0f} - ${stations_with_census['mean_household_income'].max():,.0f}")
        print(f"  - Poverty rate range: {stations_with_census['poverty_rate'].min():.1f}% - {stations_with_census['poverty_rate'].max():.1f}%")
        print(f"  - Population density range: {stations_with_census['population_density'].min():,.0f} - {stations_with_census['population_density'].max():,.0f}/km²")
        print(f"  - College educated range: {stations_with_census['percent_college_educated'].min():.1f}% - {stations_with_census['percent_college_educated'].max():.1f}%")
        print(f"  - Unemployment rate range: {stations_with_census['unemployment_rate'].min():.1f}% - {stations_with_census['unemployment_rate'].max():.1f}%")
        print(f"  - Median home value range: ${stations_with_census['median_home_value'].min():,.0f} - ${stations_with_census['median_home_value'].max():,.0f}")
        print(f"  - EJ communities identified: {stations_with_census['in_ej_community'].sum()}")
        print(f"  - Total economic indicators: {len([c for c in stations_with_census.columns if any(x in c for x in ['income', 'poverty', 'density', 'college', 'unemployment', 'home', 'economic'])])}")
    
    return stations_with_census


def load_massgis_layers(stations_gdf, ma_crs="EPSG:26986", use_census_data=True):
    """
    Load GIS layers from folders OR use Census/economic data as alternative.
    
    If use_census_data=True, fetches readily available Census/economic data instead
    of relying on sparse GIS layers.
    
    stations_gdf: GeoDataFrame of monitoring stations
    ma_crs: CRS to convert layers to
    use_census_data: If True, use Census API data instead of GIS layers
    """
    layers = {}
    
    if use_census_data:
        print("Using Census/economic data instead of GIS layers (more reliable)")
        # Convert stations to DataFrame for census function
        stations_df = pd.DataFrame({
            'station_id': stations_gdf.get('station_id', range(len(stations_gdf))),
            'latitude': stations_gdf.geometry.y if hasattr(stations_gdf.geometry, 'y') else None,
            'longitude': stations_gdf.geometry.x if hasattr(stations_gdf.geometry, 'x') else None
        })
        
        # Try to get lat/lon from existing columns
        if stations_df['latitude'].isna().all():
            # Try to extract from geometry if it's in WGS84
            try:
                stations_wgs = stations_gdf.to_crs('EPSG:4326')
                stations_df['latitude'] = stations_wgs.geometry.y
                stations_df['longitude'] = stations_wgs.geometry.x
            except:
                pass
        
        if not stations_df['latitude'].isna().all():
            stations_with_census = fetch_census_data_for_stations(stations_df, use_dor_data=True)
            layers['census_data'] = stations_with_census
            return layers
        else:
            print("Warning: Could not extract coordinates. Falling back to GIS layers.")
    
    # Original GIS loading code (fallback)
    # Mapping layer name -> folder
    layer_folders = {
        'ej_communities': 'ej2020',
        'landuse': 'Land Cover Land Use (2016) Index',
        'protected': 'openspace'
    }

    for layer_name, folder in layer_folders.items():
        try:
            # Find the .shp file in the folder
            shp_files = [f for f in os.listdir(folder) if f.endswith('.shp')]
            if not shp_files:
                raise FileNotFoundError(f"No .shp file found in {folder}")
            
            shp_path = os.path.join(folder, shp_files[0])
            gdf = gpd.read_file(shp_path).to_crs(ma_crs)
            
            # Process landuse layer - map shp_link to categories
            if layer_name == 'landuse' and 'shp_link' in gdf.columns:
                gdf['landuse'] = gdf['shp_link'].apply(map_landuse_categories)
                print(f"✓ Mapped {gdf['landuse'].nunique()} land use categories")
            
            # Process EJ layer - create vulnerability score if missing
            if layer_name == 'ej_communities':
                if 'vulnerability_score' not in gdf.columns:
                    # Create a simple vulnerability score based on available columns
                    # Check for common EJ indicator columns
                    if 'MINORITY' in gdf.columns or 'minority' in gdf.columns:
                        minority_col = 'MINORITY' if 'MINORITY' in gdf.columns else 'minority'
                        income_col = None
                        if 'LOW_INCOME' in gdf.columns:
                            income_col = 'LOW_INCOME'
                        elif 'low_income' in gdf.columns:
                            income_col = 'low_income'
                        
                        # Create composite vulnerability score
                        gdf['vulnerability_score'] = 0.5
                        if minority_col in gdf.columns:
                            gdf['vulnerability_score'] += 0.3 * (gdf[minority_col] > 0).astype(int)
                        if income_col and income_col in gdf.columns:
                            gdf['vulnerability_score'] += 0.2 * (gdf[income_col] > 0).astype(int)
                        gdf['vulnerability_score'] = gdf['vulnerability_score'].clip(0, 1)
                    else:
                        # Default vulnerability score
                        gdf['vulnerability_score'] = 0.5
            
            layers[layer_name] = gdf
            print(f"✓ Loaded {layer_name} layer from {shp_path}")

        except Exception as e:
            print(f"⚠ Could not load {layer_name} layer: {e}")
            # Fallback to mock data
            if layer_name == 'ej_communities':
                layers[layer_name] = create_mock_ej_layer(stations_gdf)
            elif layer_name == 'landuse':
                layers[layer_name] = create_mock_landuse_layer(stations_gdf)
            elif layer_name == 'protected':
                layers[layer_name] = create_mock_protected_layer(stations_gdf)
    
    return layers


In [15]:
gis_layers = load_massgis_layers(stations_gdf, use_census_data=True)
print(f"\nLoaded {len(gis_layers)} data layers")
if 'census_data' in gis_layers:
    print("✓ Using Census/economic data (more reliable than sparse GIS layers)")

Using Census/economic data instead of GIS layers (more reliable)
Attempting to use MA DOR municipal income data...
Fetching MA DOR income data for year 2019...
Loaded income data for 26 municipalities
  DOR data join returned no matches, falling back to synthetic data
Using synthetic economic proxies based on location...


NameError: name 'stations_with_census' is not defined

In [ ]:
gis_layers.keys()

## 4. Spatial Analysis and Feature Engineering

### 4.1 Calculate Land Characteristics Around Each Station

In [ ]:
def calculate_buffer_statistics(stations, layers, buffer_distances=[500, 1000, 2000, 5000], use_census_data=True):
    """
    Calculate spatial statistics for stations using either GIS layers OR Census/economic data.
    
    If use_census_data=True and census_data is in layers, uses economic indicators instead
    of land use percentages (which may be sparse/zero).
    """
    stations_with_features = stations.copy()
    
    # Check if we're using census data instead of GIS
    if use_census_data and 'census_data' in layers:
        print("Using Census/economic data for spatial features (more reliable than sparse GIS)")
        census_df = layers['census_data']
        
        # Merge census data to stations
        if 'station_id' in stations_with_features.columns and 'station_id' in census_df.columns:
            stations_with_features = stations_with_features.merge(
                census_df.drop(columns=['latitude', 'longitude'], errors='ignore'),
                on='station_id',
                how='left'
            )
        else:
            # Merge by index if no station_id
            for col in census_df.columns:
                if col not in ['station_id', 'latitude', 'longitude']:
                    stations_with_features[col] = census_df[col].values
        
        # Create derived features from economic data
        # These serve as proxies for land use patterns
        
        # Urbanization proxy (higher income + density = more urban)
        if 'median_household_income' in stations_with_features.columns:
            stations_with_features['urbanization_proxy'] = (
                (stations_with_features['median_household_income'] / 80000) * 0.5 +
                (stations_with_features.get('population_density', 0) / 5000) * 0.5
            ).clip(0, 1)
        
        # Natural cover proxy (inverse of urbanization, higher in rural areas)
        if 'dist_from_boston_km' in stations_with_features.columns:
            stations_with_features['natural_cover_proxy'] = (
                1 - np.exp(-stations_with_features['dist_from_boston_km'] / 50)
            ).clip(0, 1)
        
        # EJ vulnerability (already calculated in census function)
        if 'ej_vulnerability_proxy' in stations_with_features.columns:
            stations_with_features['ej_vulnerability'] = stations_with_features['ej_vulnerability_proxy']
        
        print(f"✓ Added {len([c for c in stations_with_features.columns if 'proxy' in c or 'income' in c or 'poverty' in c])} economic/spatial proxy features")
        return stations_with_features
    
    # Original GIS-based buffer statistics (fallback)
    # Get CRS from stations
    ma_crs = stations.crs
    
    for distance in buffer_distances:
        print(f"\nCalculating statistics for {distance}m buffer...")
        
        # Create buffers around stations
        stations_buffered = stations.copy()
        stations_buffered['buffer_geom'] = stations_buffered.geometry.buffer(distance)
        
        # Calculate land use composition
        if 'landuse' in layers:
            # Create buffer GeoDataFrame with proper geometry
            buffer_gdf = gpd.GeoDataFrame(
                stations_buffered[['station_id']],
                geometry=stations_buffered['buffer_geom'],
                crs=ma_crs
            )
            
            # Use overlay for geometric intersection and clipping
            try:
                landuse_intersect = gpd.overlay(
                    buffer_gdf,
                    layers['landuse'][['geometry', 'landuse']], 
                    how='intersection',
                    keep_geom_type=False 
                )
                
                # Calculate the area of the clipped geometry
                landuse_intersect['intersect_area_sq_m'] = landuse_intersect.geometry.area
                
                # Calculate land use percentages for each category
                for landuse_type in ['Forest', 'Urban', 'Industrial', 'Wetland', 'Agriculture']:
                    # Filter by landuse type and group by station_id
                    landuse_filtered = landuse_intersect[landuse_intersect['landuse'] == landuse_type]
                    
                    if len(landuse_filtered) > 0:
                        landuse_area = (
                            landuse_filtered
                            .groupby('station_id')['intersect_area_sq_m']
                            .sum()
                        )
                        
                        # Calculate the theoretical total area of a perfect circle buffer
                        total_area = np.pi * distance ** 2
                        
                        # Calculate percentage
                        landuse_pct = (landuse_area / total_area * 100).reset_index()
                        landuse_pct.columns = ['station_id', f'{landuse_type.lower()}_{distance}m_pct']
                        stations_with_features = stations_with_features.merge(landuse_pct, on='station_id', how='left')
                    else:
                        # No data for this landuse type - set to 0
                        stations_with_features[f'{landuse_type.lower()}_{distance}m_pct'] = 0
                        
            except Exception as e:
                print(f"  Warning: Error calculating landuse for {distance}m buffer: {e}")
                # Set default values
                for landuse_type in ['Forest', 'Urban', 'Industrial', 'Wetland', 'Agriculture']:
                    stations_with_features[f'{landuse_type.lower()}_{distance}m_pct'] = 0
        
        # Calculate impervious surface percentage (if available)
        if 'impervious' in layers:
            # Similar approach for impervious surfaces
            buffer_gdf = gpd.GeoDataFrame(
                stations_buffered[['station_id']],
                geometry=stations_buffered['buffer_geom'],
                crs=ma_crs
            )
            try:
                imperv_intersect = gpd.overlay(
                    buffer_gdf,
                    layers['impervious'][['geometry', 'imperv_pct']],
                    how='intersection',
                    keep_geom_type=False
                )
                imperv_intersect['intersect_area_sq_m'] = imperv_intersect.geometry.area
                total_area = np.pi * distance ** 2
                
                # Weighted average impervious percentage
                imperv_weighted = (
                    imperv_intersect.groupby('station_id')
                    .apply(lambda x: (x['intersect_area_sq_m'] * x['imperv_pct']).sum() / x['intersect_area_sq_m'].sum())
                    .reset_index()
                )
                imperv_weighted.columns = ['station_id', f'imperv_mean_{distance}m']
                stations_with_features = stations_with_features.merge(imperv_weighted, on='station_id', how='left')
            except Exception as e:
                print(f"  Warning: Error calculating impervious for {distance}m buffer: {e}")
        
        # Check if within EJ community (only need to do once, not per buffer)
        if 'ej_communities' in layers and distance == buffer_distances[0]:
            stations_points = stations[['station_id', 'geometry']].copy()
            try:
                ej_joined = gpd.sjoin(stations_points, layers['ej_communities'], predicate='within', how='left')
                
                if 'vulnerability_score' in ej_joined.columns:
                    ej_flags = ej_joined.groupby('station_id')['vulnerability_score'].max().reset_index()
                    ej_flags.columns = ['station_id', 'ej_vulnerability']
                    ej_flags['in_ej_community'] = (ej_flags['ej_vulnerability'] > 0).astype(int)
                else:
                    # If no vulnerability_score, just mark if in EJ community
                    ej_flags = ej_joined.groupby('station_id').size().reset_index()
                    ej_flags.columns = ['station_id', 'ej_count']
                    ej_flags['in_ej_community'] = (ej_flags['ej_count'] > 0).astype(int)
                    ej_flags['ej_vulnerability'] = ej_flags['in_ej_community'] * 0.5
                    ej_flags = ej_flags[['station_id', 'ej_vulnerability', 'in_ej_community']]
                
                stations_with_features = stations_with_features.merge(ej_flags, on='station_id', how='left')
                stations_with_features['in_ej_community'] = stations_with_features['in_ej_community'].fillna(0)
                stations_with_features['ej_vulnerability'] = stations_with_features['ej_vulnerability'].fillna(0)
            except Exception as e:
                print(f"  Warning: Error calculating EJ communities: {e}")
                stations_with_features['in_ej_community'] = 0
                stations_with_features['ej_vulnerability'] = 0
    
    # Fill NaN values with 0 for land use percentages
    landuse_cols = [col for col in stations_with_features.columns if '_pct' in col or 'imperv_' in col]
    for col in landuse_cols:
        if col in stations_with_features.columns:
            stations_with_features[col] = stations_with_features[col].fillna(0)
    
    return stations_with_features

# Calculate spatial features
stations_with_spatial = calculate_buffer_statistics(stations_gdf, gis_layers, use_census_data=True)
print(f"\nTotal features after spatial analysis: {len(stations_with_spatial.columns)}")
economic_features = [col for col in stations_with_spatial.columns if any(x in col for x in ['income', 'poverty', 'density', 'proxy', 'ej'])]
if economic_features:
    print(f"Sample economic/spatial features: {economic_features[:5]}")
else:
    gis_features = [col for col in stations_with_spatial.columns if '500m' in col]
    print(f"Sample GIS features: {gis_features[:5]}")

## 5. Water Stress from Land Use Score (WSLUS) Calculation

### 5.1 Define the Multi-Factor Risk Scoring System

In [ ]:
class WaterStressScorer:
    """
    Calculate Water Stress from Land Use Score (WSLUS)
    A unified metric combining water quality indicators with spatial land characteristics
    """
    
    def __init__(self, weights=None):
        # Default weights for different stress factors
        self.weights = weights or {
            'conductivity_stress': 0.25,
            'tds_stress': 0.20,
            'ph_stress': 0.15,
            'temperature_stress': 0.15,
            'impervious_stress': 0.15,
            'landuse_stress': 0.10
        }
        
        # Thresholds for water quality parameters (based on EPA guidelines)
        self.thresholds = {
            'conductivity': {'low': 150, 'moderate': 500, 'high': 1500},  # μS/cm
            'tds': {'low': 100, 'moderate': 300, 'high': 500},  # mg/L
            'ph': {'low': 6.5, 'optimal': 7.5, 'high': 8.5},
            'temperature': {'low': 10, 'moderate': 20, 'high': 25},  # °C
            'do': {'critical': 4, 'low': 6, 'good': 8}  # mg/L
        }
    
    def _get_column(self, df, possible_names):
        """Helper to find column with case-insensitive matching"""
        for name in possible_names:
            # Exact match
            if name in df.columns:
                return df[name]
            # Case-insensitive match
            matches = [col for col in df.columns if col.lower() == name.lower()]
            if matches:
                return df[matches[0]]
        return None
    
    def calculate_conductivity_stress(self, df):
        """Calculate stress from conductivity (indicator of salt/runoff)"""
        cond = self._get_column(df, ['conductivity_mean', 'conductivity'])
        if cond is None:
            return pd.Series(0, index=df.index)
        stress = pd.Series(0.0, index=df.index)
        
        # Low stress
        stress[cond < self.thresholds['conductivity']['low']] = 0.2
        # Moderate stress
        stress[(cond >= self.thresholds['conductivity']['low']) & 
               (cond < self.thresholds['conductivity']['moderate'])] = 0.5
        # High stress
        stress[(cond >= self.thresholds['conductivity']['moderate']) & 
               (cond < self.thresholds['conductivity']['high'])] = 0.8
        # Severe stress
        stress[cond >= self.thresholds['conductivity']['high']] = 1.0
        
        # Amplify by variability (high CV indicates instability)
        cond_cv = self._get_column(df, ['conductivity_cv', 'conductivity_std'])
        if cond_cv is not None:
            cond_mean = self._get_column(df, ['conductivity_mean', 'conductivity'])
            if cond_mean is not None and cond_mean.mean() > 0:
                cv = cond_cv.fillna(0) / cond_mean.fillna(1)
                stress = stress * (1 + cv * 0.3)
        
        return stress.clip(0, 1)
    
    def calculate_tds_stress(self, df):
        """Calculate stress from Total Dissolved Solids"""
        tds = self._get_column(df, ['TDS_mean', 'tds_mean', 'TDS', 'tds'])
        if tds is None:
            return pd.Series(0, index=df.index)
        stress = pd.Series(0.0, index=df.index)
        
        stress[tds < self.thresholds['tds']['low']] = 0.2
        stress[(tds >= self.thresholds['tds']['low']) & 
               (tds < self.thresholds['tds']['moderate'])] = 0.5
        stress[(tds >= self.thresholds['tds']['moderate']) & 
               (tds < self.thresholds['tds']['high'])] = 0.8
        stress[tds >= self.thresholds['tds']['high']] = 1.0
        
        return stress.clip(0, 1)
    
    def calculate_ph_stress(self, df):
        """Calculate stress from pH deviation from optimal range"""
        ph = self._get_column(df, ['pH_mean', 'ph_mean', 'pH', 'ph'])
        if ph is None:
            return pd.Series(0, index=df.index)
        optimal = self.thresholds['ph']['optimal']
        
        # Calculate deviation from optimal pH
        deviation = np.abs(ph - optimal)
        stress = deviation / 2.0  # Normalize to 0-1 scale
        
        # Add extreme pH stress
        stress[(ph < self.thresholds['ph']['low']) | 
               (ph > self.thresholds['ph']['high'])] = 1.0
        
        return stress.clip(0, 1)
    
    def calculate_temperature_stress(self, df):
        """Calculate thermal stress"""
        temp = self._get_column(df, ['temperature_mean', 'temperature', 'temp_mean', 'temp'])
        if temp is None:
            return pd.Series(0, index=df.index)
        stress = pd.Series(0.0, index=df.index)
        
        # Cold stress
        stress[temp < self.thresholds['temperature']['low']] = 0.3
        # Optimal range
        stress[(temp >= self.thresholds['temperature']['low']) & 
               (temp < self.thresholds['temperature']['moderate'])] = 0.1
        # Warm stress
        stress[(temp >= self.thresholds['temperature']['moderate']) & 
               (temp < self.thresholds['temperature']['high'])] = 0.6
        # Heat stress
        stress[temp >= self.thresholds['temperature']['high']] = 1.0
        
        return stress.clip(0, 1)
    
    def calculate_impervious_stress(self, df):
        """Calculate stress from impervious surface coverage"""
        imperv_cols = [col for col in df.columns if 'imperv_mean' in col]
        if not imperv_cols:
            return pd.Series(0, index=df.index)
        
        # Use the 1000m buffer as primary indicator
        if 'imperv_mean_1000m' in df.columns:
            imperv_pct = df['imperv_mean_1000m']
        else:
            imperv_pct = df[imperv_cols[0]]
        
        # Stress increases exponentially with impervious coverage
        stress = (imperv_pct / 100) ** 1.5
        
        return stress.clip(0, 1)
    
    def calculate_landuse_stress(self, df):
        """Calculate stress from problematic land uses"""
        stress = pd.Series(0.0, index=df.index)
        
        # Industrial land use is highest stress
        if 'industrial_1000m_pct' in df.columns:
            stress += df['industrial_1000m_pct'] / 100 * 0.8
        
        # Urban land use is moderate stress
        if 'urban_1000m_pct' in df.columns:
            stress += df['urban_1000m_pct'] / 100 * 0.5
        
        # Forest cover reduces stress (negative contribution)
        if 'forest_1000m_pct' in df.columns:
            stress -= df['forest_1000m_pct'] / 100 * 0.3
        
        # Wetlands also reduce stress
        if 'wetland_1000m_pct' in df.columns:
            stress -= df['wetland_1000m_pct'] / 100 * 0.2
        
        return stress.clip(0, 1)
    
    def calculate_wslus(self, df):
        """
        Calculate the composite Water Stress from Land Use Score
        """
        # Calculate individual stress components
        stress_components = {
            'conductivity_stress': self.calculate_conductivity_stress(df),
            'tds_stress': self.calculate_tds_stress(df),
            'ph_stress': self.calculate_ph_stress(df),
            'temperature_stress': self.calculate_temperature_stress(df),
            'impervious_stress': self.calculate_impervious_stress(df),
            'landuse_stress': self.calculate_landuse_stress(df)
        }
        
        # Add stress components to dataframe
        for name, values in stress_components.items():
            df[name] = values
        
        # Calculate weighted composite score
        wslus = pd.Series(0.0, index=df.index)
        for component, weight in self.weights.items():
            if component in stress_components:
                wslus += stress_components[component] * weight
        
        # Apply EJ community amplifier
        if 'ej_vulnerability' in df.columns:
            ej_amplifier = 1 + df['ej_vulnerability'] * 0.3
            wslus = wslus * ej_amplifier
        
        # Normalize to 0-100 scale
        wslus = (wslus * 100).clip(0, 100)
        
        df['WSLUS'] = wslus
        
        # Add risk categories
        df['risk_category'] = pd.cut(wslus, 
                                     bins=[0, 25, 50, 75, 100],
                                     labels=['Low', 'Moderate', 'High', 'Critical'])
        
        return df

# Initialize scorer and calculate WSLUS
scorer = WaterStressScorer()
stations_with_scores = scorer.calculate_wslus(stations_with_spatial.copy())

print("Water Stress from Land Use Scores calculated successfully!")
print(f"\nWSLUS Statistics:")
print(stations_with_scores['WSLUS'].describe())
print(f"\nRisk Category Distribution:")
print(stations_with_scores['risk_category'].value_counts())

## 6. Preservation Opportunity Identification

### 6.1 Calculate Preservation Impact Scores

In [ ]:
def identify_preservation_opportunities(df, layers):
    """
    Identify specific land parcels where preservation would have maximum impact
    """
    opportunities = []
    
    # Get CRS from dataframe
    ma_crs = df.crs
    
    # Focus on stations with elevated stress (lower threshold to find more opportunities)
    # Use 25th percentile or 30 as threshold (whichever is lower) to ensure we find opportunities
    wslus_threshold = min(df['WSLUS'].quantile(0.75), 40)  # Top 25% or 40, whichever is lower
    high_stress_stations = df[df['WSLUS'] > wslus_threshold].copy()
    
    print(f"Using WSLUS threshold: {wslus_threshold:.1f} (found {len(high_stress_stations)} stations)")
    
    if len(high_stress_stations) == 0:
        # If still no stations, use median or even lower threshold
        wslus_threshold = max(df['WSLUS'].median(), 25)
        high_stress_stations = df[df['WSLUS'] > wslus_threshold].copy()
        print(f"Lowered threshold to {wslus_threshold:.1f} (found {len(high_stress_stations)} stations)")
    
    if len(high_stress_stations) == 0:
        print("Warning: No stations above threshold. Creating opportunities based on all stations.")
        # Use all stations if still none found
        high_stress_stations = df.copy()
        print(f"Using all {len(high_stress_stations)} stations for opportunity identification")
    
    # Check if we have GIS layers or need to use alternative approach
    has_landuse = 'landuse' in layers and len(layers.get('landuse', [])) > 0
    has_protected = 'protected' in layers and len(layers.get('protected', [])) > 0
    
    if not has_landuse:
        print("No landuse GIS layer available. Creating opportunities based on station characteristics and economic data.")
        # Create opportunities based on station data and economic indicators
        for idx, station in high_stress_stations.iterrows():
            # Create opportunity based on station characteristics
            wslus = station.get('WSLUS', 0)
            ej_flag = station.get('in_ej_community', 0) == 1 or station.get('ej_vulnerability', 0) > 0.5
            
            # Estimate area based on urbanization proxy or distance from Boston
            if 'urbanization_proxy' in station.index:
                area_hectares = 50 + (1 - station['urbanization_proxy']) * 100  # More area in rural areas
            elif 'dist_from_boston_km' in station.index:
                area_hectares = 50 + min(station['dist_from_boston_km'] / 10, 100)
            else:
                area_hectares = 75  # Default
            
            # Calculate preservation score
            preservation_score = wslus * 0.6 + (ej_flag * 30) + (area_hectares / 10)
            estimated_impact = preservation_score * area_hectares
            
            opportunities.append({
                'rank': len(opportunities) + 1,
                'station_id': station.get('station_id', f'ST{idx}'),
                'landuse_type': 'Priority Area',  # Generic since no GIS data
                'area_hectares': area_hectares,
                'preservation_score': preservation_score,
                'estimated_impact': estimated_impact,
                'wslus_score': wslus,
                'ej_priority': ej_flag,
                'parcel_geometry': station.geometry.buffer(500)  # 500m buffer as opportunity area
            })
        
        if len(opportunities) > 0:
            opp_df = pd.DataFrame(opportunities)
            opp_df = opp_df.sort_values('preservation_score', ascending=False).reset_index(drop=True)
            opp_df['rank'] = range(1, len(opp_df) + 1)
            print(f"Created {len(opp_df)} opportunities based on station characteristics")
            return opp_df
    
    # Original GIS-based approach (if landuse layer is available)
    for idx, station in high_stress_stations.iterrows():
        # Create search area (2km upstream buffer)
        search_buffer = station.geometry.buffer(2000)
        
        # Find unprotected land parcels
        if has_landuse:
            # Get land parcels in search area
            search_gdf = gpd.GeoDataFrame([{'geometry': search_buffer}], crs=ma_crs)
            nearby_land = gpd.sjoin(layers['landuse'], search_gdf, predicate='intersects')
            
            # Exclude already protected lands
            try:
                protected_geoms = layers['protected'].unary_union
                # Check if geometries intersect with protected areas
                unprotected = nearby_land[~nearby_land.geometry.intersects(protected_geoms)]
            except Exception as e:
                print(f"  Warning: Could not filter protected lands: {e}")
                unprotected = nearby_land
            
            # Calculate preservation value for each parcel
            for land_idx, parcel in unprotected.iterrows():
                # Calculate distance to station
                distance = station.geometry.distance(parcel.geometry.centroid)
                
                # Calculate preservation score
                preservation_score = 0
                
                # Higher score for forests and wetlands
                if parcel['landuse'] in ['Forest', 'Wetland']:
                    preservation_score += 40
                elif parcel['landuse'] == 'Agriculture':
                    preservation_score += 20
                
                # Proximity bonus (closer = higher impact)
                proximity_score = max(0, 30 * (1 - distance / 2000))
                preservation_score += proximity_score
                
                # EJ community bonus
                if station['in_ej_community'] == 1:
                    preservation_score += 20
                
                # Water stress bonus
                stress_bonus = station['WSLUS'] / 100 * 30
                preservation_score += stress_bonus
                
                opportunities.append({
                    'station_id': station['station_id'],
                    'parcel_geometry': parcel.geometry,
                    'landuse_type': parcel['landuse'],
                    'area_hectares': parcel.geometry.area / 10000,
                    'distance_to_station': distance,
                    'preservation_score': preservation_score,
                    'station_wslus': station['WSLUS'],
                    'in_ej_community': station['in_ej_community'],
                    'estimated_impact': preservation_score * parcel.geometry.area / 10000  # Score × area
                })
    
    opportunities_df = pd.DataFrame(opportunities)
    
    # Rank opportunities
    if len(opportunities_df) > 0:
        opportunities_df = opportunities_df.sort_values('estimated_impact', ascending=False)
        opportunities_df['rank'] = range(1, len(opportunities_df) + 1)
    
    return opportunities_df

# Identify preservation opportunities
preservation_opportunities = identify_preservation_opportunities(stations_with_scores, gis_layers)

print(f"Identified {len(preservation_opportunities)} preservation opportunities")
if len(preservation_opportunities) > 0:
    print(f"\nTop 10 Preservation Opportunities:")
    print(preservation_opportunities[['rank', 'landuse_type', 'area_hectares', 
                                     'preservation_score', 'estimated_impact']].head(10))

## 7. Model Performance Analysis

### 7.1 Clustering Analysis to Identify Stress Patterns

In [ ]:
def perform_clustering_analysis(df):
    """
    Identify spatial clusters of water stress
    """
    # Select features for clustering
    feature_cols = ['conductivity_stress', 'tds_stress', 'ph_stress', 
                   'temperature_stress', 'impervious_stress', 'landuse_stress']
    
    X = df[feature_cols].fillna(0)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Determine optimal number of clusters using elbow method
    inertias = []
    silhouette_scores = []
    K_range = range(2, min(10, len(df)))
    
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    
    # Use optimal k (highest silhouette score)
    optimal_k = K_range[np.argmax(silhouette_scores)]
    
    # Final clustering
    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    df['stress_cluster'] = kmeans.fit_predict(X_scaled)
    
    # Calculate cluster centroids in original scale
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
    cluster_profiles = pd.DataFrame(cluster_centers, columns=feature_cols)
    cluster_profiles['cluster_id'] = range(optimal_k)
    
    # Spatial clustering using DBSCAN
    coords = df[['geometry']].copy()
    coords['x'] = coords.geometry.x
    coords['y'] = coords.geometry.y
    
    # DBSCAN for spatial hotspot detection
    dbscan = DBSCAN(eps=3000, min_samples=3)  # 3km radius, min 3 stations
    df['spatial_cluster'] = dbscan.fit_predict(coords[['x', 'y']])
    
    # Identify hotspots (spatial clusters with high WSLUS)
    hotspot_stats = df[df['spatial_cluster'] != -1].groupby('spatial_cluster').agg({
        'WSLUS': ['mean', 'max', 'count'],
        'station_id': 'count'
    })
    
    return df, cluster_profiles, hotspot_stats

# Perform clustering
stations_clustered, cluster_profiles, hotspot_stats = perform_clustering_analysis(stations_with_scores)

print(f"Identified {cluster_profiles.shape[0]} stress pattern clusters")
print(f"\nCluster Profiles:")
print(cluster_profiles.round(3))

if len(hotspot_stats) > 0:
    print(f"\nIdentified {len(hotspot_stats)} spatial hotspots")
    print(hotspot_stats)

In [ ]:
### 7.1.5 Feature Analysis and Simplification

# Analyze and simplify features before modeling
def analyze_and_simplify_features(df, target_col='WSLUS', missing_threshold=0.5, 
                                   variance_threshold=1e-6, correlation_threshold=0.01):
    """
    Analyze features and remove low-quality ones that hurt model performance
    """
    print("="*60)
    print("FEATURE ANALYSIS AND SIMPLIFICATION")
    print("="*60)
    
    # Get potential feature columns (exclude geometry, IDs, target)
    exclude_cols = ['geometry', 'station_id', target_col, 'risk_category', 'stress_cluster', 'spatial_cluster']
    potential_features = [col for col in df.columns 
                         if col not in exclude_cols 
                         and df[col].dtype in [np.float64, np.int64, np.float32, np.int32]]
    
    print(f"\nInitial potential features: {len(potential_features)}")
    
    # Check if target exists
    if target_col not in df.columns:
        print(f"Warning: Target column '{target_col}' not found!")
        return df, []
    
    # Filter out rows with missing target
    df_clean = df.dropna(subset=[target_col]).copy()
    print(f"Stations with valid {target_col}: {len(df_clean)}")
    
    # Step 1: Check missing values
    print("\n1. Checking missing values...")
    missing_stats = {}
    for col in potential_features:
        if col in df_clean.columns:
            missing_pct = df_clean[col].isna().sum() / len(df_clean)
            missing_stats[col] = missing_pct
    
    high_missing = [col for col, pct in missing_stats.items() if pct > missing_threshold]
    if high_missing:
        print(f"   Removing {len(high_missing)} features with >{missing_threshold*100}% missing:")
        for col in high_missing[:10]:  # Show first 10
            print(f"     - {col}: {missing_stats[col]:.1%} missing")
        if len(high_missing) > 10:
            print(f"     ... and {len(high_missing)-10} more")
        potential_features = [f for f in potential_features if f not in high_missing]
    
    # Step 2: Check variance
    print("\n2. Checking feature variance...")
    variance_stats = {}
    for col in potential_features:
        if col in df_clean.columns:
            var = df_clean[col].var()
            variance_stats[col] = var
    
    low_variance = [col for col, var in variance_stats.items() if var < variance_threshold]
    if low_variance:
        print(f"   Removing {len(low_variance)} low-variance features:")
        for col in low_variance[:10]:
            print(f"     - {col}: variance = {variance_stats[col]:.2e}")
        if len(low_variance) > 10:
            print(f"     ... and {len(low_variance)-10} more")
        potential_features = [f for f in potential_features if f not in low_variance]
    
    # Step 3: Check correlation with target
    print("\n3. Checking correlation with target...")
    correlations = {}
    for col in potential_features:
        if col in df_clean.columns:
            # Fill missing with median for correlation calculation
            col_data = df_clean[col].fillna(df_clean[col].median())
            target_data = df_clean[target_col]
            # Remove rows where either is still NaN
            valid_mask = ~(col_data.isna() | target_data.isna())
            if valid_mask.sum() > 10:
                corr = np.abs(np.corrcoef(col_data[valid_mask], target_data[valid_mask])[0, 1])
                correlations[col] = corr if not np.isnan(corr) else 0
            else:
                correlations[col] = 0
    
    low_correlation = [col for col, corr in correlations.items() if corr < correlation_threshold]
    if low_correlation:
        print(f"   Removing {len(low_correlation)} features with |correlation| < {correlation_threshold}:")
        for col in sorted(low_correlation, key=lambda x: correlations[x])[:10]:
            print(f"     - {col}: |corr| = {correlations[col]:.4f}")
        if len(low_correlation) > 10:
            print(f"     ... and {len(low_correlation)-10} more")
        potential_features = [f for f in potential_features if f not in low_correlation]
    
    # Step 4: Check for constant or near-constant features
    print("\n4. Checking for constant features...")
    constant_features = []
    for col in potential_features:
        if col in df_clean.columns:
            unique_vals = df_clean[col].nunique()
            if unique_vals <= 1:
                constant_features.append(col)
    
    if constant_features:
        print(f"   Removing {len(constant_features)} constant features")
        potential_features = [f for f in potential_features if f not in constant_features]
    
    # Step 5: Analyze by feature type/category
    print("\n5. Feature summary by category:")
    feature_categories = {
        'Land Use': [f for f in potential_features if any(x in f for x in ['forest', 'urban', 'industrial', 'wetland', 'agriculture', 'water'])],
        'Composite': [f for f in potential_features if any(x in f for x in ['developed', 'natural', 'ratio', 'index', 'weighted', 'diversity'])],
        'Impervious': [f for f in potential_features if 'imperv' in f],
        'EJ': [f for f in potential_features if 'ej' in f],
        'Water Quality': [f for f in potential_features if any(x in f for x in ['ph', 'conductivity', 'tds', 'temperature', 'do'])],
        'Other': [f for f in potential_features if not any(f in cat for cat in [
            [f for f in potential_features if any(x in f for x in ['forest', 'urban', 'industrial', 'wetland', 'agriculture', 'water'])],
            [f for f in potential_features if any(x in f for x in ['developed', 'natural', 'ratio', 'index', 'weighted', 'diversity'])],
            [f for f in potential_features if 'imperv' in f],
            [f for f in potential_features if 'ej' in f],
            [f for f in potential_features if any(x in f for x in ['ph', 'conductivity', 'tds', 'temperature', 'do'])]
        ])]
    }
    
    for category, features in feature_categories.items():
        if features:
            avg_corr = np.mean([correlations.get(f, 0) for f in features if f in correlations])
            print(f"   {category}: {len(features)} features (avg |corr| = {avg_corr:.3f})")
    
    # Step 6: Focus on best features by buffer distance
    print("\n6. Top features by buffer distance:")
    buffer_dists = [500, 1000, 2000, 5000]
    for dist in buffer_dists:
        dist_features = [f for f in potential_features if f'{dist}m' in f]
        if dist_features:
            dist_corrs = [(f, correlations.get(f, 0)) for f in dist_features if f in correlations]
            dist_corrs.sort(key=lambda x: x[1], reverse=True)
            if dist_corrs:
                best = dist_corrs[0]
                print(f"   {dist}m buffer: {len(dist_features)} features, best = {best[0]} (|corr|={best[1]:.3f})")
    
    print(f"\n{'='*60}")
    print(f"Final feature count: {len(potential_features)} (removed {len(df.columns) - len(potential_features) - len(exclude_cols)} low-quality features)")
    print(f"{'='*60}\n")
    
    # Store feature analysis results
    df_clean['_feature_analysis_done'] = True
    
    return df_clean, potential_features

# Analyze and simplify features
stations_analyzed, quality_features = analyze_and_simplify_features(
    stations_clustered, 
    target_col='WSLUS',
    missing_threshold=0.5,  # Remove if >50% missing
    variance_threshold=1e-6,  # Remove if variance too low
    correlation_threshold=0.01  # Remove if correlation with target too low
)

print(f"\nUsing {len(quality_features)} high-quality features for modeling")
print(f"Top 10 features by correlation:")
if 'WSLUS' in stations_analyzed.columns:
    corrs = []
    for feat in quality_features[:20]:  # Check top 20
        if feat in stations_analyzed.columns:
            data = stations_analyzed[feat].fillna(stations_analyzed[feat].median())
            target = stations_analyzed['WSLUS']
            valid = ~(data.isna() | target.isna())
            if valid.sum() > 10:
                corr = np.abs(np.corrcoef(data[valid], target[valid])[0, 1])
                if not np.isnan(corr):
                    corrs.append((feat, corr))
    corrs.sort(key=lambda x: x[1], reverse=True)
    for i, (feat, corr) in enumerate(corrs[:10], 1):
        print(f"  {i}. {feat}: |corr| = {corr:.3f}")


### 7.2 Predictive Model for Water Stress

In [ ]:
def build_predictive_model(df):
    """
    Build an improved predictive model to estimate WSLUS from land characteristics
    Uses feature selection, hyperparameter tuning, and better data cleaning
    """
    # Use pre-analyzed quality features if available, otherwise select features
    if '_feature_analysis_done' in df.columns:
        # Use the quality features from analysis (exclude metadata columns)
        exclude_cols = ['geometry', 'station_id', 'WSLUS', 'risk_category', 'stress_cluster', 
                       'spatial_cluster', '_feature_analysis_done']
        feature_cols = [col for col in df.columns 
                       if col not in exclude_cols 
                       and df[col].dtype in [np.float64, np.int64, np.float32, np.int32]]
        print(f"Using pre-analyzed features: {len(feature_cols)} features")
    else:
        # Fallback: select features manually (simplified - focus on most important)
        print("Warning: Using fallback feature selection. Run feature analysis first for better results.")
        # Focus on most predictive feature types
        spatial_keywords = ['forest_', 'urban_', 'developed_', 'natural_', 'ej_']
        feature_cols = [col for col in df.columns if any(x in col for x in spatial_keywords)]
        
        # Include water quality metrics if available
        wq_keywords = ['_mean', '_median']
        wq_features = [col for col in df.columns if any(x in col for x in wq_keywords) 
                       and any(p in col.lower() for p in ['ph', 'conductivity', 'tds', 'temperature', 'do'])]
        feature_cols.extend(wq_features)
        
        # Remove geometry and non-numeric columns
        feature_cols = [col for col in feature_cols if col in df.columns 
                       and col != 'geometry' and df[col].dtype in [np.float64, np.int64, np.float32, np.int32]]
    
    if len(feature_cols) == 0:
        print("No spatial features available for modeling")
        return None, None, None
    
    # Filter out rows with NaN WSLUS values
    df_clean = df.dropna(subset=['WSLUS']).copy()
    
    if len(df_clean) == 0:
        print("No valid WSLUS values available for modeling")
        return None, None, None
    
    # Remove outliers in WSLUS (keep 95% of data)
    q5 = df_clean['WSLUS'].quantile(0.05)
    q95 = df_clean['WSLUS'].quantile(0.95)
    df_clean = df_clean[(df_clean['WSLUS'] >= q5) & (df_clean['WSLUS'] <= q95)]
    
    print(f"Using {len(df_clean)} stations with valid WSLUS scores (dropped {len(df) - len(df_clean)} with NaN/outliers)")
    
    # Prepare features
    X = df_clean[feature_cols].copy()
    
    # Fill missing values with median (better than 0 for many features)
    for col in X.columns:
        if X[col].isna().sum() > 0:
            median_val = X[col].median()
            X[col] = X[col].fillna(median_val if not pd.isna(median_val) else 0)
    
    # Remove features with zero variance or very low variance
    feature_variance = X.var()
    low_variance_features = feature_variance[feature_variance < 1e-6].index.tolist()
    if low_variance_features:
        print(f"Removing {len(low_variance_features)} low-variance features")
        X = X.drop(columns=low_variance_features)
        feature_cols = [f for f in feature_cols if f not in low_variance_features]
    
    y = df_clean['WSLUS']
    
    # Remove outliers in features using IQR method
    for col in X.columns:
        Q1 = X[col].quantile(0.25)
        Q3 = X[col].quantile(0.75)
        IQR = Q3 - Q1
        if IQR > 0:
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR
            X[col] = X[col].clip(lower=lower_bound, upper=upper_bound)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=None)
    
    # Feature selection using mutual information (better for non-linear relationships)
    print(f"\nOriginal features: {len(feature_cols)}")
    selector = SelectKBest(score_func=mutual_info_regression, k=min(30, len(feature_cols)))
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    
    selected_features = [feature_cols[i] for i in selector.get_support(indices=True)]
    print(f"Selected {len(selected_features)} features using mutual information")
    
    # Scale features using RobustScaler (less sensitive to outliers)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train_selected)
    X_test_scaled = scaler.transform(X_test_selected)
    
    # Hyperparameter tuning for Random Forest
    print("\nTuning Random Forest hyperparameters...")
    param_grid_rf = {
        'n_estimators': [100, 200],
        'max_depth': [10, 15, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    
    rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf_grid = GridSearchCV(rf_base, param_grid_rf, cv=5, scoring='r2', n_jobs=-1, verbose=0)
    rf_grid.fit(X_train_scaled, y_train)
    rf_model = rf_grid.best_estimator_
    
    print(f"Best RF params: {rf_grid.best_params_}")
    
    # Predictions
    y_pred_train = rf_model.predict(X_train_scaled)
    y_pred_test = rf_model.predict(X_test_scaled)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation
    cv_scores = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    print("\n=== Predictive Model Performance ===")
    print(f"Training R²: {train_r2:.3f}")
    print(f"Test R²: {test_r2:.3f}")
    print(f"Training RMSE: {train_rmse:.2f}")
    print(f"Test RMSE: {test_rmse:.2f}")
    print(f"Training MAE: {train_mae:.2f}")
    print(f"Test MAE: {test_mae:.2f}")
    print(f"Cross-validation R² (mean ± std): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': rf_model.feature_importances_,
        'mi_score': selector.scores_[selector.get_support()]
    }).sort_values('importance', ascending=False)
    
    # Try Gradient Boosting with tuning
    print("\nTuning Gradient Boosting hyperparameters...")
    param_grid_gb = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.05, 0.1]
    }
    
    gb_base = GradientBoostingRegressor(random_state=42)
    gb_grid = GridSearchCV(gb_base, param_grid_gb, cv=5, scoring='r2', n_jobs=-1, verbose=0)
    gb_grid.fit(X_train_scaled, y_train)
    gb_model = gb_grid.best_estimator_
    gb_test_r2 = r2_score(y_test, gb_model.predict(X_test_scaled))
    
    print(f"Best GB params: {gb_grid.best_params_}")
    print(f"Gradient Boosting Test R²: {gb_test_r2:.3f}")
    
    # Use the better model
    if gb_test_r2 > test_r2:
        print(f"\nUsing Gradient Boosting model (R² = {gb_test_r2:.3f} vs RF R² = {test_r2:.3f})")
        best_model = gb_model
    else:
        print(f"\nUsing Random Forest model (R² = {test_r2:.3f} vs GB R² = {gb_test_r2:.3f})")
        best_model = rf_model
    
    return best_model, feature_importance, {
        'train_r2': train_r2, 'test_r2': test_r2, 
        'train_rmse': train_rmse, 'test_rmse': test_rmse,
        'train_mae': train_mae, 'test_mae': test_mae,
        'cv_scores': cv_scores,
        'selected_features': selected_features,
        'scaler': scaler,
        'selector': selector
    }

# Build and evaluate model using simplified features
model, feature_importance, metrics = build_predictive_model(stations_analyzed)

if feature_importance is not None:
    print("\n=== Top 10 Most Important Features ===")
    print(feature_importance.head(10))

## 8. Visualization and Reporting

### 8.1 Create Interactive Map

In [ ]:
def create_interactive_map(stations_df, opportunities_df=None):
    """
    Create an interactive Folium map showing WSLUS scores and preservation opportunities
    """
    # Convert to WGS84 for web mapping
    stations_wgs = stations_df.to_crs('EPSG:4326')
    
    # Calculate map center
    center_lat = stations_wgs.geometry.y.mean()
    center_lon = stations_wgs.geometry.x.mean()
    
    # Create base map
    m = folium.Map(location=[center_lat, center_lon], 
                   zoom_start=10,
                   tiles='OpenStreetMap')
    
    # Add tile layers
    folium.TileLayer('CartoDB positron', name='Light Mode').add_to(m)
    folium.TileLayer('CartoDB dark_matter', name='Dark Mode').add_to(m)
    
    # Color scheme for WSLUS scores
    def get_color(wslus):
        if wslus < 25:
            return 'green'
        elif wslus < 50:
            return 'yellow'
        elif wslus < 75:
            return 'orange'
        else:
            return 'red'
    
    # Add monitoring stations
    station_group = folium.FeatureGroup(name='Monitoring Stations')
    
    for idx, row in stations_wgs.iterrows():
        # Skip rows with NaN WSLUS or invalid coordinates
        if pd.isna(row.get('WSLUS')) or pd.isna(row.geometry.y) or pd.isna(row.geometry.x):
            continue
            
        # Create popup text
        wslus_val = row.get('WSLUS', 0)
        risk_cat = row.get('risk_category', 'Unknown')
        station_id = row.get('station_id', 'N/A')
        popup_text = f"""
        <div style="font-family: Arial; width: 250px;">
            <h4>Station {station_id}</h4>
            <b>WSLUS Score:</b> {wslus_val:.1f}<br>
            <b>Risk Category:</b> {risk_cat}<br>
            <hr>
            <b>Stress Components:</b><br>
            • Conductivity: {row.get('conductivity_stress', 0):.2f}<br>
            • TDS: {row.get('tds_stress', 0):.2f}<br>
            • pH: {row.get('ph_stress', 0):.2f}<br>
            • Temperature: {row.get('temperature_stress', 0):.2f}<br>
            • Impervious: {row.get('impervious_stress', 0):.2f}<br>
            • Land Use: {row.get('landuse_stress', 0):.2f}<br>
            <hr>
            <b>EJ Community:</b> {'Yes' if row.get('in_ej_community', 0) == 1 else 'No'}<br>
        </div>
        """
        
        # Add marker
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=8 + (wslus_val/10 if pd.notna(wslus_val) else 5),  # Size based on score
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"Station {station_id}: {wslus_val:.1f}",
            color='black',
            fillColor=get_color(wslus_val) if pd.notna(wslus_val) else 'gray',
            fillOpacity=0.8,
            weight=2
        ).add_to(station_group)
    
    station_group.add_to(m)
    
    # Add preservation opportunities if available
    if opportunities_df is not None and len(opportunities_df) > 0:
        opp_group = folium.FeatureGroup(name='Top Preservation Opportunities')
        
        # Show top 20 opportunities
        for idx, row in opportunities_df.head(20).iterrows():
            # Convert geometry to WGS84
            parcel_crs = stations_df.crs  # Get CRS from stations
            geom_wgs = gpd.GeoSeries([row['parcel_geometry']], crs=parcel_crs).to_crs('EPSG:4326')[0]
            
            popup_text = f"""
            <div style="font-family: Arial;">
                <h4>Preservation Opportunity #{row['rank']}</h4>
                <b>Land Type:</b> {row['landuse_type']}<br>
                <b>Area:</b> {row['area_hectares']:.1f} hectares<br>
                <b>Preservation Score:</b> {row['preservation_score']:.1f}<br>
                <b>Impact Score:</b> {row['estimated_impact']:.0f}<br>
            </div>
            """
            
            folium.GeoJson(
                geom_wgs.__geo_interface__,
                style_function=lambda x: {
                    'fillColor': 'purple',
                    'color': 'purple',
                    'weight': 2,
                    'fillOpacity': 0.3
                },
                tooltip=f"Opportunity #{row['rank']}",
                popup=folium.Popup(popup_text, max_width=300)
            ).add_to(opp_group)
        
        opp_group.add_to(m)
    
    # Add heatmap layer (filter out NaN values)
    heat_data = [[row.geometry.y, row.geometry.x, row['WSLUS']] 
                 for idx, row in stations_wgs.iterrows()
                 if pd.notna(row.get('WSLUS')) and pd.notna(row.geometry.y) and pd.notna(row.geometry.x)]
    
    if len(heat_data) > 0:
        plugins.HeatMap(heat_data, name='WSLUS Heatmap', 
                       radius=30, blur=20).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 180px; height: 140px; 
                background-color: white; z-index: 1000; 
                border:2px solid grey; border-radius: 5px;
                font-size: 14px; font-family: Arial;
                ">
        <p style="margin: 10px;"><b>WSLUS Score</b></p>
        <p style="margin: 10px;"><span style="color: green;">●</span> Low (0-25)</p>
        <p style="margin: 10px;"><span style="color: gold;">●</span> Moderate (25-50)</p>
        <p style="margin: 10px;"><span style="color: orange;">●</span> High (50-75)</p>
        <p style="margin: 10px;"><span style="color: red;">●</span> Critical (75-100)</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Create map
interactive_map = create_interactive_map(stations_clustered, preservation_opportunities)
print("Interactive map created successfully!")

# Save map
interactive_map.save('watershed_preservation_map.html')
print("Map saved as 'watershed_preservation_map.html'")

# Display map (if in Jupyter)
interactive_map

### 8.2 Generate Summary Dashboard

In [ ]:
def create_summary_dashboard(stations_df, opportunities_df):
    """
    Create comprehensive visualizations for the analysis
    """
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('WSLUS Score Distribution', 'Risk Categories',
                       'Stress Components by Station', 'Top Preservation Opportunities',
                       'WSLUS vs Impervious Coverage', 'Spatial Clustering'),
        specs=[[{'type': 'histogram'}, {'type': 'pie'}],
               [{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'scatter'}]]
    )
    
    # 1. WSLUS Distribution
    fig.add_trace(
        go.Histogram(x=stations_df['WSLUS'], nbinsx=20, name='WSLUS'),
        row=1, col=1
    )
    
    # 2. Risk Categories Pie Chart
    risk_counts = stations_df['risk_category'].value_counts()
    fig.add_trace(
        go.Pie(labels=risk_counts.index, values=risk_counts.values,
               marker=dict(colors=['green', 'yellow', 'orange', 'red'])),
        row=1, col=2
    )
    
    # 3. Stress Components
    stress_cols = ['conductivity_stress', 'tds_stress', 'ph_stress', 
                  'temperature_stress', 'impervious_stress', 'landuse_stress']
    mean_stress = stations_df[stress_cols].mean()
    fig.add_trace(
        go.Bar(x=stress_cols, y=mean_stress.values, name='Mean Stress'),
        row=2, col=1
    )
    
    # 4. Top Preservation Opportunities
    if len(opportunities_df) > 0:
        top_opps = opportunities_df.head(10)
        fig.add_trace(
            go.Bar(x=top_opps['rank'], y=top_opps['estimated_impact'],
                  name='Impact Score'),
            row=2, col=2
        )
    
    # 5. WSLUS vs Impervious Coverage
    if 'imperv_mean_1000m' in stations_df.columns:
        fig.add_trace(
            go.Scatter(x=stations_df['imperv_mean_1000m'], 
                      y=stations_df['WSLUS'],
                      mode='markers',
                      marker=dict(color=stations_df['WSLUS'], colorscale='RdYlGn_r'),
                      name='Stations'),
            row=3, col=1
        )
    
    # 6. Spatial Clustering
    if 'stress_cluster' in stations_df.columns:
        fig.add_trace(
            go.Scatter(x=stations_df.geometry.x, 
                      y=stations_df.geometry.y,
                      mode='markers',
                      marker=dict(color=stations_df['stress_cluster'], 
                                size=stations_df['WSLUS']/5,
                                colorscale='Viridis'),
                      name='Clusters'),
            row=3, col=2
        )
    
    # Update layout
    fig.update_layout(
        title_text="Watershed Preservation Opportunity Analysis Dashboard",
        showlegend=False,
        height=1200,
        width=1400
    )
    
    # Update axes
    fig.update_xaxes(title_text="WSLUS Score", row=1, col=1)
    fig.update_xaxes(title_text="Stress Component", row=2, col=1)
    fig.update_xaxes(title_text="Opportunity Rank", row=2, col=2)
    fig.update_xaxes(title_text="Impervious %", row=3, col=1)
    fig.update_xaxes(title_text="X Coordinate", row=3, col=2)
    
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Mean Stress", row=2, col=1)
    fig.update_yaxes(title_text="Impact Score", row=2, col=2)
    fig.update_yaxes(title_text="WSLUS Score", row=3, col=1)
    fig.update_yaxes(title_text="Y Coordinate", row=3, col=2)
    
    return fig

# Create dashboard
dashboard = create_summary_dashboard(stations_clustered, preservation_opportunities)
dashboard.show()

# Save dashboard
dashboard.write_html('watershed_analysis_dashboard.html')
print("Dashboard saved as 'watershed_analysis_dashboard.html'")

## 9. Key Insights and Recommendations

### 9.1 Generate Actionable Report

In [ ]:
def generate_executive_summary(stations_df, opportunities_df, model_metrics):
    """
    Generate executive summary with key findings and recommendations
    """
    summary = {
        'overview': {},
        'key_findings': [],
        'critical_areas': [],
        'recommendations': [],
        'impact_estimates': {}
    }
    
    # Overview statistics
    summary['overview'] = {
        'total_stations': len(stations_df),
        'mean_wslus': stations_df['WSLUS'].mean(),
        'critical_stations': len(stations_df[stations_df['risk_category'] == 'Critical']),
        'high_risk_stations': len(stations_df[stations_df['risk_category'].isin(['High', 'Critical'])]),
        'ej_affected_stations': stations_df['in_ej_community'].sum(),
        'model_accuracy': model_metrics['test_r2'] if model_metrics else None
    }
    
    # Key findings
    # Finding 1: Primary stress drivers
    stress_cols = ['conductivity_stress', 'tds_stress', 'ph_stress', 
                  'temperature_stress', 'impervious_stress', 'landuse_stress']
    mean_stress = stations_df[stress_cols].mean().sort_values(ascending=False)
    summary['key_findings'].append(
        f"Primary stress driver: {mean_stress.index[0].replace('_stress', '').title()} "
        f"(mean stress: {mean_stress.values[0]:.2f})"
    )
    
    # Finding 2: EJ impact
    ej_stations = stations_df[stations_df['in_ej_community'] == 1]
    if len(ej_stations) > 0:
        ej_mean_wslus = ej_stations['WSLUS'].mean()
        non_ej_mean_wslus = stations_df[stations_df['in_ej_community'] == 0]['WSLUS'].mean()
        summary['key_findings'].append(
            f"EJ communities experience {(ej_mean_wslus/non_ej_mean_wslus - 1)*100:.1f}% "
            f"higher water stress than non-EJ areas"
        )
    
    # Finding 3: Impervious surface correlation
    if 'imperv_mean_1000m' in stations_df.columns:
        corr = stations_df[['WSLUS', 'imperv_mean_1000m']].corr().iloc[0, 1]
        summary['key_findings'].append(
            f"Strong correlation (r={corr:.2f}) between impervious surface and water stress"
        )
    
    # Critical areas
    critical_stations = stations_df[stations_df['risk_category'] == 'Critical'].sort_values('WSLUS', ascending=False)
    for idx, station in critical_stations.head(5).iterrows():
        summary['critical_areas'].append({
            'station_id': station['station_id'],
            'wslus_score': station['WSLUS'],
            'primary_issue': max([(station[col], col) for col in stress_cols])[1].replace('_stress', ''),
            'in_ej_community': bool(station['in_ej_community'])
        })
    
    # Recommendations
    if len(opportunities_df) > 0:
        top_opportunities = opportunities_df.head(10)
        total_area = top_opportunities['area_hectares'].sum()
        total_impact = top_opportunities['estimated_impact'].sum()
        
        summary['recommendations'].append(
            f"Preserve {total_area:.0f} hectares across {len(top_opportunities)} priority parcels"
        )
        summary['recommendations'].append(
            f"Focus on {'forest and wetland' if 'Forest' in top_opportunities['landuse_type'].values else 'natural'} "
            f"preservation for maximum impact"
        )
    
    # Targeted interventions
    if mean_stress.values[0] > 0.6:  # High conductivity stress
        summary['recommendations'].append(
            "Implement road salt reduction program and enhanced stormwater management"
        )
    
    if stations_df['impervious_stress'].mean() > 0.5:
        summary['recommendations'].append(
            "Retrofit impervious surfaces with green infrastructure in high-stress watersheds"
        )
    
    # Impact estimates
    if len(opportunities_df) > 0:
        summary['impact_estimates'] = {
            'parcels_to_preserve': len(opportunities_df.head(20)),
            'total_preservation_area_ha': opportunities_df.head(20)['area_hectares'].sum(),
            'estimated_wslus_reduction': 15.0,  # Placeholder - would need modeling
            'affected_population': stations_df['in_ej_community'].sum() * 5000,  # Rough estimate
            'priority_watersheds': len(stations_df['spatial_cluster'].unique())
        }
    
    return summary

# Generate summary
executive_summary = generate_executive_summary(stations_clustered, preservation_opportunities, metrics)

print("\n" + "="*60)
print("EXECUTIVE SUMMARY: WATERSHED PRESERVATION OPPORTUNITY MAP")
print("="*60)

print("\nOVERVIEW:")
for key, value in executive_summary['overview'].items():
    print(f"  • {key.replace('_', ' ').title()}: {value:.1f if isinstance(value, float) else value}")

print("\nKEY FINDINGS:")
for finding in executive_summary['key_findings']:
    print(f"  • {finding}")

print("\nCRITICAL AREAS REQUIRING IMMEDIATE ATTENTION:")
for area in executive_summary['critical_areas']:
    ej_flag = " [EJ Community]" if area['in_ej_community'] else ""
    print(f"  • Station {area['station_id']}: WSLUS={area['wslus_score']:.1f}, "
          f"Primary issue: {area['primary_issue']}{ej_flag}")

print("\nRECOMMENDATIONS:")
for i, rec in enumerate(executive_summary['recommendations'], 1):
    print(f"  {i}. {rec}")

if executive_summary['impact_estimates']:
    print("\nESTIMATED IMPACT OF INTERVENTIONS:")
    for key, value in executive_summary['impact_estimates'].items():
        print(f"  • {key.replace('_', ' ').title()}: {value:.1f if isinstance(value, float) else value}")

print("\n" + "="*60)

## 10. Export Results and Next Steps

### 10.1 Save Analysis Results

In [ ]:
# Save processed data
stations_clustered.to_csv('stations_with_wslus_scores.csv', index=False)
print("✓ Saved station analysis to 'stations_with_wslus_scores.csv'")

if len(preservation_opportunities) > 0:
    preservation_opportunities.to_csv('preservation_opportunities.csv', index=False)
    print("✓ Saved preservation opportunities to 'preservation_opportunities.csv'")

# Save model
if model is not None:
    import joblib
    joblib.dump(model, 'wslus_predictive_model.pkl')
    print("✓ Saved predictive model to 'wslus_predictive_model.pkl'")

# Save summary as JSON
with open('executive_summary.json', 'w') as f:
    json.dump(executive_summary, f, indent=2, default=str)
print("✓ Saved executive summary to 'executive_summary.json'")

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)
print("\nGenerated outputs:")
print("  1. watershed_preservation_map.html - Interactive map")
print("  2. watershed_analysis_dashboard.html - Analysis dashboard")
print("  3. stations_with_wslus_scores.csv - Complete station analysis")
print("  4. preservation_opportunities.csv - Ranked preservation targets")
print("  5. wslus_predictive_model.pkl - ML model for predictions")
print("  6. executive_summary.json - Key findings and recommendations")

### 10.2 Next Steps and Deployment Recommendations

## Next Steps for Implementation:

### 1. **Data Integration**
   - Connect to live MassGIS data feeds
   - Integrate real-time water quality monitoring data
   - Add precipitation and storm event data for temporal analysis

### 2. **Model Refinement**
   - Calibrate thresholds using Massachusetts-specific water quality standards
   - Incorporate seasonal variations and storm response patterns
   - Add economic valuation of preservation benefits

### 3. **Web Application Development**
   - Build interactive dashboard using Dash or Streamlit
   - Create API endpoints for real-time WSLUS calculations
   - Implement user authentication for different stakeholder groups

### 4. **Stakeholder Engagement**
   - Present to MassDEP, conservation organizations, and EJ communities
   - Gather feedback and refine scoring methodology
   - Develop training materials and documentation

### 5. **Policy Integration**
   - Align with Massachusetts watershed management plans
   - Support grant applications with quantitative preservation metrics
   - Create regular reporting mechanisms for tracking progress

### 6. **Expansion Opportunities**
   - Scale to cover entire Massachusetts watershed network
   - Integrate climate change projections
   - Add cost-benefit analysis for specific interventions
   - Develop mobile app for field data collection

## Technical Deployment Architecture:

```
┌─────────────┐     ┌──────────────┐     ┌───────────────┐
│ Water       │────▶│ ETL Pipeline │────▶│ PostgreSQL/   │
│ Quality     │     │ (Airflow)    │     │ PostGIS DB    │
│ Sensors     │     └──────────────┘     └───────────────┘
└─────────────┘              │                    │
                            │                    │
┌─────────────┐              ▼                    ▼
│ MassGIS     │────▶┌──────────────┐     ┌───────────────┐
│ Data Feeds  │     │ WSLUS Model  │────▶│ Web Dashboard │
└─────────────┘     │ (Python API) │     │ (React/Dash)  │
                   └──────────────┘     └───────────────┘
                            │                    │
┌─────────────┐              ▼                    ▼
│ Weather     │────▶┌──────────────┐     ┌───────────────┐
│ Data        │     │ ML Pipeline  │────▶│ Mobile App    │
└─────────────┘     │ (MLflow)     │     │ (React Native)│
                   └──────────────┘     └───────────────┘
```

This notebook provides the foundation for a powerful decision support system that bridges the gap between water quality monitoring and land conservation priorities!